In [ ]:
import pandas as pd
import os 
import numpy as np 
import tensorflow as tf
import likelihood as lh
from tqdm.notebook import tqdm 

### Apply the likelihood model 

The following code works on a directory `inference_results` which contains (pickled) files, each of these containing the output of 
the inference model on a given location. Each file should be an ndarray of shape `(m, dim_x, dim_y)` where `m` is the number of images corresponding to this location (typically approximately 100). 

In [ ]:
results_dir = 'inference_results'
results = {}

for i, loc in tqdm(enumerate(os.listdir(results_dir))):
        
    with open(os.path.join(results_dir, loc), 'rb') as f: 
        probs = pickle.load(f)
        
    probs = tf.convert_to_tensor(probs, dtype=float)
    probs = tf.reshape(
        probs, [probs.shape[0], probs.shape[1], probs.shape[2], 1]
    )
    
    # kernel smoothing 
    smth = lh.tf_smooth(probs, 'box', size=3)
    
    # Apply model 
    seed = int(loc.split('_')[-1].split('.p')[0])
    ts = lh.change_ts(smth, learning_rate=0.5, num_steps=4000, 
                      seed=seed, return_model=False)  
    results[loc] = ts 
        
    # save every 20 iterations 
    if i % 20 == 0: 
        with open('results.p', 'wb') as f: 
            pickle.dump(results, f)

with open('results.p', 'wb') as f: 
    pickle.dump(results, f)